## Question splitting

This notebook takes folders containing full transcriptions of the interviews, and uses openai's chat completion API with a 1-shot prompting strategy to separate the full text into separate blocks for each of the (7 to 9) survey questions. 

The main functions for the processing are contained in the file utils.py. 

Outputs: 
1. Folders with one txt file for each interview, containing the LLM output (blocks separated by LLM delimiters and reasoning steps):
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_groundtruth_round1
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1
2. Datasets in wide format (1 line per interview) with question text and answer for each question:
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_groundtruth_round1.xlsx
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1.xlsx
    - ../../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1.xlsx

Note: for round 1 of the interviews, data was prepared in long format (1 line for each question), for reasons relevant at the time. The splitting function has been tailored to this shape, and at the end we reshape into wide format (1 line per interview/phone number). For round 2 we replicated this extra (unnecessary) step to be able to use the processing functions from round 1, but this should be simplified in future work. 

In [1]:
# Imports and constants for the notebook
%reload_ext autoreload
%autoreload 2
import utils
import pandas as pd
import os

from utils import SYSTEM_PROMPT_SPLIT

# Set directory paths
data_dir = "../../data"
transcripts_dir = os.path.join(data_dir, "transcriptions_all")
qualtrics_dir = os.path.join(data_dir, "working/qualtrics")
output_dir_splits = os.path.join(transcripts_dir, "splits")
transcriptions_round_1_automatic = os.path.join(transcripts_dir, "transcriptions_round1_relabeled")
transcriptions_round_2_automatic = os.path.join(transcripts_dir, "transcriptions_round2_relabeled")


## Set input file paths

transcriptions_round_1_dta_path = os.path.join(transcripts_dir, "full_transcriptions_with_controls_11apr2024.dta")
transcriptions_round_1_groundtruth_path = os.path.join(transcripts_dir, "transcriptions_all_split_11apr2024.xlsx")
controls_round_2_path = os.path.join(qualtrics_dir, "qualtrics_v2_07may.dta")

# examples to use for 1-shot prompting
examples_round_1_path = os.path.join(transcripts_dir, "examples_wide_round1.xlsx")
examples_round_2_path = os.path.join(transcripts_dir, "examples_wide_round2.xlsx")

## Set output file paths and directories

output_dir_splits_round1_groundtruth = os.path.join(output_dir_splits, '1-shot_gpt-4o_groundtruth_round1')
output_dataset_splits_round1_groundtruth = os.path.join(output_dir_splits, '1-shot_gpt-4o_groundtruth_round1.xlsx')

output_dir_splits_round1_nova2 = os.path.join(output_dir_splits, '1-shot_gpt-4o_nova2_round1')
output_dataset_splits_round1_nova2 = os.path.join(output_dir_splits, '1-shot_gpt-4o_nova2_round1.xlsx')

output_dir_splits_round2_nova2 = os.path.join(output_dir_splits, '1-shot_gpt-4o_nova2_round2')
output_dataset_splits_round2_nova2 = os.path.join(output_dir_splits, '1-shot_gpt-4o_nova2_round2.xlsx')




### Split round 1, ground truth

In [2]:
df = pd.read_excel(transcriptions_round_1_groundtruth_path)

# drop if missing reference_phone
df = df.dropna(subset=['reference_phone'])

# make reference_phone into int
df['reference_phone'] = df['reference_phone'].astype(int)

# reshape dataset to wide format
df_wide = utils.reshape_dataset(df)
df_examples = pd.read_excel(examples_round_1_path)

# generate dictionary of examples
examples = {}
for _, row in df_examples.iterrows():
    if row['winner'] == 1:
        examples['winner'] = utils.produce_example(row)
    else:
        examples['loser'] = utils.produce_example(row)


In [26]:
df_wide.head()


,reference_phone,winner,full_transcription,question_id_0,question_id_1,question_id_2,question_id_3,question_id_4,question_id_5,question_id_6,...,question_answer_0,question_answer_1,question_answer_2,question_answer_3,question_answer_4,question_answer_5,question_answer_6,question_answer_7,question_answer_8,file_name
0,233534645992,0.0,Audio Name: 20240213-092637_3534645992_Phoebe_...,demographics,self_intro,money_daily,financial_situation,health_general,alcohol,suggestions,...,,,,,,,,,,233534645992
1,233550272383,0.0,"C:Hello\nA: Hello, good afternoon. \nC: Aftern...",demographics,self_intro,money_daily,management_learn,health_general,alcohol,suggestions,...,,,,,,,,,,233550272383
2,233240964575,0.0,Audio Name: -102901_3240964575_Phoebe_U2024021...,demographics,self_intro,money_daily,management_learn,health_general,happiness,suggestions,...,,,,,,,,,,233240964575
3,233549157270,0.0,Audio Name: 20240213-084111_3549157270_Phoebe_...,demographics,self_intro,money_daily,saving,health_general,happiness,suggestions,...,,,,,,,,,,233549157270
4,233247992954,0.0,Audio Name: 20240213-094656_3247992954_Phoebe_...,demographics,self_intro,money_daily,financial_situation,health_general,stress,suggestions,...,,,,,,,,,,233247992954


In [7]:
df_examples.head()

,file_name,winner,full_transcription,step1,step2,reasoning,question_id_0,question_id_1,question_id_2,question_id_3,...,question_text_8,question_answer_0,question_answer_1,question_answer_2,question_answer_3,question_answer_4,question_answer_5,question_answer_6,question_answer_7,question_answer_8
0,20240222-094955_3549049712_SamDe_USTAWI-all,0,Caller: Hello.\nAgent: Good morning.\nCaller: ...,"<STEP1>\nBeginning of block 1: Agent: Okay, fi...",<STEP2>\nLast sentence of the preamble: Caller...,<REASONING>\n1. The number of blocks detected ...,demographics,self_intro,money_daily,saving,...,NaN,Caller: Hello.\nAgent: Good morning.\nCaller: ...,"Agent: Okay, fine. That's okay. Alright. pleas...","Agent: Okay. So, we want to know about your fi...","Agent: Oh, okay. Okay. That's fine. That's fi...","Agent: Okay. So, please, we want to talk about...","Agent: Okay. So, what are some of the things t...","Agent: Okay. Okay. All right. That's fine. So,...","Agent: Okay, alright, thank you very much for ...",NaN
1,Audio name: 20240221-145817_3559385272_Rose_US...,1,A: Hello good afternoon\nC: Good afternoon\nA:...,"<STEP1>\nBeginning of block 1: A: Okay, all ri...",<STEP2>\nLast sentence of the preamble: A: Oka...,<REASONING>\n1. The number of blocks detected ...,demographics,self_intro,money_daily,financial_situation,...,NaN,A: Hello good afternoon\nC: Good afternoon\nA:...,"A: Okay, all right. So please, now we move to ...","A: Okay. Okay. So please, I will now ask you s...","A: Okay. Okay. So, please, how can you compare...","A: A little. Okay, understood. All right. So, ...","A: Okay. All right. So, please, if you think a...",A: Yeah. Yeah. Sorry about that. All right. S...,"C: I win three times. First one was 1,000. Sec...","A: Okay. So, please, we are done with this sur..."


In [3]:
# for development: truncate dataset to one row with winner == 0 and one row with winner == 1
df_win = df_wide[df_wide['winner'] == 1].head(1)
df_lose = df_wide[df_wide['winner'] == 0].head(1)
df_wide = pd.concat([df_win, df_lose])

# set output directory and file
out_dir = output_dir_splits_round1_groundtruth
out_file = output_dataset_splits_round1_groundtruth

# split the transcriptions and save the output
utils.process_transcriptions(df_wide, '1-shot', SYSTEM_PROMPT_SPLIT, out_dir, out_file, 'gpt-4o', examples)

Processing rows:  50%|█████     | 1/2 [01:37<01:37, 97.12s/it]

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_groundtruth_round1/split_233548065056.txt


Processing rows: 100%|██████████| 2/2 [02:42<00:00, 81.16s/it]

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_groundtruth_round1/split_233534645992.txt
Data saved to ../data/transcriptions_all/splits/1-shot_gpt-4o_groundtruth_round1.xlsx


### Split round 1, nova2

Prepare dataset

In [8]:
## Prepare wide dataset for question splitting

# Load text files
simple_transcripts = utils.load_text_files(transcriptions_round_1_automatic)
print(len(simple_transcripts))

# Load dta data
dta_data = pd.read_stata(transcriptions_round_1_dta_path)
# convert reference_phone to int
dta_data['reference_phone'] = dta_data['reference_phone'].astype(int)

# Load excel data
df_split = pd.read_excel(transcriptions_round_1_groundtruth_path) 
df_split.columns
# drop if missing reference_phone
df_split = df_split.dropna(subset=['reference_phone'])
# convert reference_phone to int
df_split['reference_phone'] = df_split['reference_phone'].astype(int)

# merge on reference_phone
df = pd.merge(df_split, simple_transcripts, how='left', on='reference_phone')

# rename columns for compatibility with function utils.process_transcriptions
df = df.rename(columns = { 'full_transcription' : 'ground_truth'})
df = df.rename(columns = { 'transcription' : 'full_transcription'})

# Create wide dataframe
df_wide = utils.reshape_dataset(df, file_level_vars=['winner', 'ground_truth', 'full_transcription'])
df_wide.head()
print(len(df_wide))
# remove rows without transcription
df_wide = df_wide[df_wide['full_transcription'] != '']
print(len(df_wide))

78
90
58


Question split

In [9]:
# for development: truncate dataset to one row with winner == 0 and one row with winner == 1
df_win = df_wide[df_wide['winner'] == 1].head(1)
df_lose = df_wide[df_wide['winner'] == 0].head(1)
df_wide = pd.concat([df_win, df_lose])


# set output directory and file
out_dir = output_dir_splits_round1_nova2 
out_file = output_dataset_splits_round1_nova2

# split the transcriptions and save the output
utils.process_transcriptions(df_wide, '1-shot', SYSTEM_PROMPT_SPLIT, out_dir, out_file, 'gpt-4o', examples)

Processing rows:  50%|█████     | 1/2 [01:55<01:55, 115.24s/it]

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1/split_233242092383.txt


Processing rows: 100%|██████████| 2/2 [02:55<00:00, 88.00s/it] 

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1/split_233534645992.txt
Data saved to ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round1.xlsx


### Split round 2, nova2

load text

In [13]:
# Load text files into a dataframe and save as an excel file
out_path = os.path.join(transcripts_dir, "transcriptions_with_controls_round2_nova2.xlsx")
df, samples = utils.make_dataset_from_txt_and_dta(dta_path=controls_round_2_path, output_path=out_path, directory=transcriptions_round_2_automatic)
print(f"Number of transcriptions left after cleaning: {samples}.\n")

Number of transcriptions left after cleaning: 75.



Prepare dataset for splitting

In [14]:
# Step 1: define the question maps

question_map_loser = {
  0: {"id": "demographics", "text": ""},
  1: {"id": "self_intro", "text": "To begin, we would like to know you better. Can you tell me a little bit more about yourself and your family?"},
  2: {"id": "money_daily", "text": "How do you organize and keep track of your finances from day to day? Please tell me about any specific methods and tools you might use in this process."},
  # 3: Filled later based on randomizedquestionsmoney_do
  4: {"id": "health_general", "text": "How would you describe your and your family's health?"},
  # 5: Filled later based on randomizedquestionshealth_do
  6: {"id": "suggestions", "text": "How can we design a better and more popular lottery? Do you have any suggestions for us?"},
  7: {"id": "conclusion", "text": ""},
}

question_map_winner = {
  0: {"id": "demographics", "text": ""},
  1: {"id": "self_intro", "text": "To begin, we would like to know you better. Can you tell me a little bit more about yourself and your family?"},
  2: {"id": "money_daily", "text": "How do you organize and keep track of your finances from day to day? Please tell me about any specific methods and tools you might use in this process."},
  # 3: Filled later based on randomizedquestionsmoney_do
  4: {"id": "health_general", "text": "How would you describe your and your family's health?"},
  # 5: Filled later based on randomizedquestionshealth_do
  6: {"id": "suggestions", "text": "How can we design a better and more popular lottery? Do you have any suggestions for us?"},
  7: {"id": "impact", "text": "In what way has winning the raffle impacted your life? How have you spent the money, or how do you plan to spend it?"},
  8: {"id": "regret", "text": "Is there something you regret about spending your prize money?"},
  9: {"id": "recommendation", "text": "What advice would you have for someone who wins the same prize as yours today?"},
  10: {"id": "conclusion", "text": ""}
}

In [15]:
# Step 2: prepare long dataset for splitting and save as an excel file
data_path = out_path
df = utils.prepare_dataset_for_split(data_path, 11, 8, question_map_loser, question_map_winner)
df.to_excel(os.path.join(transcripts_dir, "transcriptions_round2_nova2_forsplit.xlsx"), index=False)

In [16]:
# Step 3: prepare wide dataset for splitting. 
## Note: It is inefficient to create a dataset in long format and then convert it to wide format. 
## This is how the code evolved, but should be refactored in the future.

# convert variable reference_phone to integer
df['reference_phone'] = df['reference_phone'].astype(int)

# reshape dataset to wide format
df_wide = utils.reshape_dataset(df)

# generate dictionary of examples
df_examples = pd.read_excel(examples_round_2_path)
examples = {}
for _, row in df_examples.iterrows():
    if row['winner'] == 1:
        examples['winner'] = utils.produce_example(row)
    else:
        examples['loser'] = utils.produce_example(row)


Split questions

In [18]:
# for development: truncate dataset to one row with winner == 0 and one row with winner == 1
df_win = df_wide[df_wide['winner'] == 1].head(1)
df_lose = df_wide[df_wide['winner'] == 0].head(1)
df_wide = pd.concat([df_win, df_lose])

# set output directory and file
out_dir = output_dir_splits_round2_nova2
out_file = output_dataset_splits_round2_nova2

# split the transcriptions and save the output
utils.process_transcriptions(df_wide, '1-shot', SYSTEM_PROMPT_SPLIT, out_dir, out_file, 'gpt-4o', examples, num_questions_loser=6, num_questions_winner=9)

Processing rows:  50%|█████     | 1/2 [01:17<01:17, 77.24s/it]

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round2/split_233203670791.txt


Processing rows: 100%|██████████| 2/2 [02:31<00:00, 75.93s/it]

output saved to  ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round2/split_233201629308.txt
Data saved to ../data/transcriptions_all/splits/1-shot_gpt-4o_nova2_round2.xlsx
